In [1]:
import anndata
import scanpy as sc
import scgen
import pandas as pd  
import numpy as np

font = {'family' : 'Arial',
        'size'   : 14}

Global seed set to 0
/Users/meiqiliu/miniforge3/envs/fa_base2/lib/python3.8/site-packages/flax/struct.py:132: FutureWarning: jax.tree_util.register_keypaths is deprecated, and will be removed in a future release. Please use `register_pytree_with_keys()` instead.
  jax.tree_util.register_keypaths(data_clz, keypaths)
/Users/meiqiliu/miniforge3/envs/fa_base2/lib/python3.8/site-packages/flax/struct.py:132: FutureWarning: jax.tree_util.register_keypaths is deprecated, and will be removed in a future release. Please use `register_pytree_with_keys()` instead.
  jax.tree_util.register_keypaths(data_clz, keypaths)


In [2]:
train = sc.read("./data/PapalexiSatija2021_eccite_RNA.h5ad")
train.layers['counts'] = train.X.copy()
sc.pp.log1p(train)

sc.pp.highly_variable_genes(train, n_top_genes=100, subset=True)
train.X = train.layers['counts']
train.X.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 3., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [3]:
def sample_obs(adata,size_n):
    """
    params 
    -------------
    sample for all control and stim pairs 
    size_n:the number of rows we will consider in comparison
    output x,y in sparse matrix form
    which is sample1 and sample2 drawn from two specified codition
    """

    x = adata.X
    #print(x.shape)(16893, 6998)
    n_rows=x.shape[0]
    
    df = pd.DataFrame({'x':np.arange(n_rows)})
    #shuffle the data index
    x_sample1=df['x'].sample(frac=1, replace=False).values[:size_n]
    x_sample2=df['x'].sample(frac=1, replace=False).values[:size_n]
    
    return x[x_sample1,:], x[x_sample2,:]

import random

def subset(alist, idxs):
    '''
        use idxs to subset alist
        alist: list
        idxs: list
    '''
    sub_list = []
    for idx in idxs:
        sub_list.append(alist[idx])

    return sub_list

def split_list(alist, group_num=3, shuffle=True, retain_left=False):
    '''
        split data into 3 subset and let each subset contains the len(alist)//group number of elements
        shuffle: whether shuffle the splitted data, default: True
        retain_left: if list alist is splited into the number of group_num subset and there is some element remain，
        whether take the remaining elements as a subset
    '''

    index = list(range(len(alist))) 

    
    if shuffle: 
        random.shuffle(index) 
    
    elem_num = len(alist) // group_num 
    sub_lists = {}
    
   
    for idx in range(group_num):
        start, end = idx*elem_num, (idx+1)*elem_num
        sub_lists[str(idx)] = subset(alist, index[start:end])
    
  
    if retain_left and group_num * elem_num != len(index): 
        sub_lists[str(idx+1)] = subset(alist, index[end:])
    
    return sub_lists



def sample_control_control(adata,size_n):
    """
    the sampling for ctrl and ctrl
    split the data into three samples
    shuffle the three samples
    return two data set have maximum between-sample distance
    
    """
    x = adata.X
    n_rows=x.shape[0]
    index_dict=split_list(range(n_rows))# in form {0: [1,2,3],1:[3,44,2...],3:[]}
    sample1=x[index_dict['0'],:size_n].toarray()
    sample2=x[index_dict['1'],:size_n].toarray()
    sample3=x[index_dict['2'],:size_n].toarray() 
    
    pairs={}
    pairs[0]=(sample1,sample2)
    pairs[1]=(sample1,sample3)
    pairs[2]=(sample2,sample3)
    return pairs 
        

In [4]:

def data_prep(adata,conditions=None,sample_ctrl=False):
    """
    param
    -----------
    adata
    return_mean: bool variable True if we want to compute mean of sampled data per cell to
    find statistic between mean of data from sample1 and sample2
    output: the sampled data from sample1 and sample2 of type array
    """
    control = adata[adata.obs["perturbation"] == conditions["x"]]
    stim = adata[adata.obs["perturbation"] == conditions["y"]]
    
    #fix the number of rows of sampled data as the minimum number of rows between sample1,sample2
    n=np.minimum(control.shape[0],stim.shape[0])
    
    x,_ = sample_obs(control,n)
    y,_ = sample_obs(stim,n)
        
    x=x.toarray()
    y=y.toarray()
        
    
    return x,y
 
def compute_from_mean(x,y,fn,if_return=False):
    """
    param
    -------
    x,y:data from sample1 and sample2
    fn: the statistic function with input x,y
    print statistic computed with mean of sample 1 and 2
    """ 
    
    x_mean = np.mean(x, axis=0).ravel()
    y_mean = np.mean(y, axis=0).ravel()
    mean = fn(x_mean,y_mean)
#     print("statistic computed with mean of sample 1 and 2:",mean)
    if if_return:
        return mean

def compute_from_sample(x,y,fn):
    """
    param
    -----------
    x,y:sampled data from sample1 and sample2 in sparse matric form
    output: the average of statistic computed between each data from sample1 and sample2
    """
#     x=x.toarray()
#     y=y.toarray()
    a=dict.fromkeys(range(x.shape[0]))
    
    for i in range(x.shape[0]):
        a[i]=fn(x[i],y[i])
        
    return np.mean(list(a.values()))

def dist_based(x,y):
    """
    transpose the data so that we can compute static between genes (columns)
    """
    m=np.minimum(x.shape[0],y.shape[0])
    x=x.T[:,:m]#delete .toarray()
    y=y.T[:,:m]
    return x,y


def test(fn,train,metric_str):
    """
    fn: the function for computing specific statistic to apply 
    print out the test result for statistic between (stim,ctrl) and (ctrl,ctrl)
    
    """
    list_stim=list(train.obs['perturbation'].unique())
    list_stim.remove('control')
    conditions={"x":"control","y":"stim"}
    
    difference=[]
    if_stim=['stim']*len(list_stim)
    
    for stim in list_stim:
        conditions["y"]=str(stim)
#         print(conditions)
#         print("mean of computed statistics:",fn(train,conditions))
        difference.append(fn(train,conditions))
    
    #lastly,for (ctrl,ctrl) where we apply a different sampling method, get three points 
    conditions["y"]="control"
    control = train[train.obs["perturbation"] == conditions["x"]]
    print(conditions)
    pairs=sample_control_control(control,control.shape[0])
    for i in range(3):
        (x,y)=pairs[i]
        print("mean of computed statistics for (contrl, control):",fn(train,conditions,True,x,y))
        #set the sample_ctrl True
        difference.append(fn(train,conditions,True,x,y))
    zero=['control']*3
    if_stim=if_stim + zero
    metric=[metric_str]*len(if_stim)
        
    return difference,if_stim,metric    
              
              

In [49]:
from scvi import settings
import torch 
def log_zinb_positive(
    x: torch.Tensor, mu: torch.Tensor, theta: torch.Tensor, pi: torch.Tensor, eps=1e-8
):
    """Log likelihood (scalar) of a minibatch according to a zinb model.
    Parameters
    ----------
    x
        Data
    mu
        mean of the negative binomial (has to be positive support) (shape: minibatch x vars)
    theta
        inverse dispersion parameter (has to be positive support) (shape: minibatch x vars)
    pi
        logit of the dropout parameter (real support) (shape: minibatch x vars)
    eps
        numerical stability constant
    Notes
    -----
    We parametrize the bernoulli using the logits, hence the softplus functions appearing.
    """
    # theta is the dispersion rate. If .ndimension() == 1, it is shared for all cells (regardless of batch or labels)
    if theta.ndimension() == 1:
        theta = theta.view(
            1, theta.size(0)
        )  # In this case, we reshape theta for broadcasting

    # Uses log(sigmoid(x)) = -softplus(-x)
    softplus_pi = F.softplus(-pi)
   
    log_theta_eps = torch.log(theta + eps)
    log_theta_mu_eps = torch.log(theta + mu + eps)
    pi_theta_log = -pi + theta * (log_theta_eps - log_theta_mu_eps)

    case_zero = F.softplus(pi_theta_log) - softplus_pi
    mul_case_zero = torch.mul((x < eps).type(torch.float32), case_zero)
#     print(mul_case_zero)

    case_non_zero = (
        -softplus_pi
        + pi_theta_log
        + x * (torch.log(mu + eps) - log_theta_mu_eps)
        + torch.lgamma(x + theta)
        - torch.lgamma(theta)
        - torch.lgamma(x + 1)
    )
    mul_case_non_zero = torch.mul((x > eps).type(torch.float32), case_non_zero)

    res = mul_case_zero + mul_case_non_zero

    return res

In [7]:
def test_get_data(zinb,train,metric_str):
    """
    fn: the function to get datasets after sampling before we fit the gene in sample1 and sample2
    
    """
    list_stim=list(train.obs['perturbation'].unique())
    list_stim.remove('control')
    conditions={"x":"control","y":"stim"}
    
    # sample1 is always sampled from control and sample2 is sampled from 99 different stim 
    #use prep_data to sample respectively 
    first = True 
    for stim in list_stim:
        conditions["y"]=str(stim)
        print(conditions)
        if first:
            print("data:",zinb(train,conditions,str(stim)))
            first=False 

    
    #lastly,for (ctrl,ctrl) where we apply a different sampling method, get three points 
    conditions["y"]="control"
    control = train[train.obs["perturbation"] == conditions["x"]]
    print(conditions)
    pairs=sample_control_control(control,control.shape[0])
    for i in range(3):
        (x,y)=pairs[i]
        print(" data for (contrl, control):",zinb(train,conditions,'control'+str(i),True,x,y))
        #set the sample_ctrl True

    #     difference.append(fn(train,conditions,True,x,y))
    # zero=['control']*3
    # if_stim=if_stim + zero
    # metric=[metric_str]*len(if_stim)
        
    return None

In [56]:
from statsmodels.discrete.count_model import (ZeroInflatedNegativeBinomialP, ZeroInflatedPoisson,
                                              ZeroInflatedGeneralizedPoisson)

import matplotlib.pyplot as plt
import statsmodels.api as sm 
from joblib import Parallel,delayed
import torch
import torch.nn.functional as F

def process_item_alpha(i,x):
    plt.hist(x[i],bins=30)
    plt.yscale('log')
    plt.show()
    
    print(all(elem == 0 for elem in x[i]))
    print("gene",i)

    if all(elem == 0 for elem in x[i]):
        return None 

    else:
        one=np.ones_like(x[i])
        model=sm.NegativeBinomialP(x[i],one).fit()
        if model.params[1]>0:
            # print(model.summary())
            return model.params[1]
        else:
            return None 
    
def process_item_const(i,x):
    plt.hist(x[i],bins=30)
    plt.yscale('log')
    plt.show()
    
    print(all(elem == 0 for elem in x[i]))
    print("gene",i)

    if all(elem == 0 for elem in x[i]):
        return None 

    else:
        one=np.ones_like(x[i])
        model=sm.NegativeBinomialP(x[i],one).fit()
        if model.params[1]>0:
        # print(model.summary())
            return model.params[0]
        else:
            return None 

def process_per_batch(x,num_from,num_to):
    x=x[num_from:num_to,:]

    alpha=Parallel(n_jobs=-1,backend="multiprocessing")(
        delayed(process_item_alpha)(i,x)for i in range(x.shape[0])
    )

    const=Parallel(n_jobs=-1,backend="multiprocessing")(
        delayed(process_item_const)(i,x)for i in range(x.shape[0])
    ) 
    return alpha,const    



def zinb(adata, conditions,stim_str,sample_ctrl=False,ctrl1=None,ctrl2=None):
    if sample_ctrl:
        x,y=ctrl1,ctrl2
    else:
        x,y=data_prep(adata,conditions)
        
    x,y=dist_based(x,y)

    np.savetxt(stim_str+'datax.csv', x, delimiter=',')
    np.savetxt(stim_str+'datay.csv', y, delimiter=',')

    alpha_x, const_x =process_per_batch(x,0,99)
    alpha_y, const_y = process_per_batch(y,0,99)

    df=pd.DataFrame({"alpha_x":alpha_x,"const_x":const_x,"alpha_y":alpha_y,"const_y":const_y})
    df.to_csv(stim_str+'df.csv', index=False)

#     p=1/(1+np.exp(const_x))*alpha_x # probability of single success 
#     #1-p probability of failure 
#     logit=np.log((1-p)/p)
    

#     x_val=log_zinb_positive(torch.Tensor(x),torch.Tensor(np.exp(const_x)),torch.Tensor(1/alpha_x),torch.Tensor(logit))
#     print(x_val)
  
    return None

def log_zinb(adata, conditions,stim_str,sample_ctrl=False,ctrl1=None,ctrl2=None):
    
    if sample_ctrl:
        x,y=ctrl1,ctrl2
    else:
        x,y=data_prep(adata,conditions)
        
    x,y=dist_based(x,y)

    output=[]
    
    for i in range(x.shape[0]):
        print(x[i].shape)
#         plt.hist(x[i],bins=30)
#         plt.yscale('log')
#         plt.show()



        if all(elem == 0 for elem in x[i]):
            continue 

        else:
            one=np.ones_like(x[i])
            model=sm.NegativeBinomialP(x[i],one).fit()
            if model.params[1]>0:
                const_x=model.params[0]
                alpha_x=model.params[1]
                mu=np.exp(const_x)
                print("mu",mu)
                theta=1/alpha_x
                p=1/(1+theta)*mu
                logit=np.log((1-p)/p)
                print("theta",theta)
                print("p",p)
                print("logit",logit)
                x_val=log_zinb_positive(torch.tensor(x[i]),torch.tensor(mu),torch.tensor(theta),torch.tensor(logit))
                unique, counts = torch.unique(x_val, return_counts=True)
                print("output:",unique)
                print("counts:",counts)
                output.append(x_val)
            else:
                continue 

  
    return None 

test_get_data(log_zinb,train,'zinb')


{'x': 'control', 'y': 'STAT2g2'}
(266,)
Optimization terminated successfully.
         Current function value: 0.024750
         Iterations: 0
         Function evaluations: 1
         Gradient evaluations: 1
mu 0.003759398496240588
theta 20.0
p 0.00017901897601145655
logit 8.627839711503304
output: tensor([-1.4215e+01, -6.7168e-07])
counts: tensor([  1, 265])
(266,)
Optimization terminated successfully.
         Current function value: 0.326571
         Iterations: 3
         Function evaluations: 10
         Gradient evaluations: 10
mu 0.1917293233082118
theta 0.028322598929980715
p 0.18644861399303625
logit 1.473253418305425
output: tensor([-9.3551, -9.1329, -8.4106, -7.5417, -6.2425, -5.4395, -0.0106])
counts: tensor([  1,   1,   1,   1,   4,   7, 251])
(266,)
Optimization terminated successfully.
         Current function value: 0.735402
         Iterations: 5
         Function evaluations: 10
         Gradient evaluations: 10
mu 1.0300751879698709
theta 0.04203289726006362
p 0.98

/Users/meiqiliu/miniforge3/envs/fa_base2/lib/python3.8/site-packages/statsmodels/base/model.py:592: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '
/Users/meiqiliu/miniforge3/envs/fa_base2/lib/python3.8/site-packages/statsmodels/discrete/discrete_model.py:3122: RuntimeWarning: invalid value encountered in log
  a1 * np.log(a1) + y * np.log(mu) -
/Users/meiqiliu/miniforge3/envs/fa_base2/lib/python3.8/site-packages/statsmodels/discrete/discrete_model.py:3159: RuntimeWarning: invalid value encountered in log
  dgterm = dgpart + np.log(a1 / a2) + 1 - a3 / a2
/Users/meiqiliu/miniforge3/envs/fa_base2/lib/python3.8/site-packages/statsmodels/discrete/discrete_model.py:3123: RuntimeWarning: invalid value encountered in log
  (y + a1) * np.log(a2))
/Users/meiqiliu/miniforge3/envs/fa_base2/lib/python3.8/site-packages/statsmodels/discrete/discrete_model.py:3122: RuntimeWarning: divide by zero encoun

         Current function value: 0.044289
         Iterations: 1
         Function evaluations: 124
         Gradient evaluations: 112
mu 0.0075187969924814775
theta 47.47727051581377
p 0.00015509942933005625
logit 8.771289055697846
output: tensor([-1.3669e+01, -1.1617e-06])
counts: tensor([  2, 264])
(266,)
Optimization terminated successfully.
         Current function value: 0.086846
         Iterations: 3
         Function evaluations: 13
         Gradient evaluations: 13
mu 0.045112781954346134
theta 0.005354809806664969
p 0.04487249826061066
logit 3.058019745056626
output: tensor([-1.1538e+01, -9.2578e+00, -8.4579e+00, -5.3596e-04])
counts: tensor([  1,   1,   1, 263])
(266,)
Optimization terminated successfully.
         Current function value: 0.284713
         Iterations: 4
         Function evaluations: 11
         Gradient evaluations: 11
mu 0.13909774436102887
theta 0.028301571743892116
p 0.13526940751936575
logit 1.8551496035344217
output: tensor([-9.1039e+00, -8.7892e+00,

/Users/meiqiliu/miniforge3/envs/fa_base2/lib/python3.8/site-packages/statsmodels/base/model.py:592: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '
/var/folders/12/d54y_pk95zdbftjg_qkcj_c80000gq/T/ipykernel_42688/534514316.py:123: RuntimeWarning: invalid value encountered in log
  logit=np.log((1-p)/p)
/Users/meiqiliu/miniforge3/envs/fa_base2/lib/python3.8/site-packages/statsmodels/discrete/discrete_model.py:3122: RuntimeWarning: invalid value encountered in log
  a1 * np.log(a1) + y * np.log(mu) -
/Users/meiqiliu/miniforge3/envs/fa_base2/lib/python3.8/site-packages/statsmodels/discrete/discrete_model.py:3123: RuntimeWarning: invalid value encountered in log
  (y + a1) * np.log(a2))
/var/folders/12/d54y_pk95zdbftjg_qkcj_c80000gq/T/ipykernel_42688/534514316.py:123: RuntimeWarning: invalid value encountered in log
  logit=np.log((1-p)/p)
/Users/meiqiliu/miniforge3/envs/fa_base2/lib/python3

output: tensor([-1.2432e+01, -1.0491e+01, -3.2736e-05])
counts: tensor([  1,   3, 791])
(795,)
Optimization terminated successfully.
         Current function value: 0.506703
         Iterations: 3
         Function evaluations: 11
         Gradient evaluations: 11
mu 0.24654088050306855
theta 0.08575782319711077
p 0.2270680212803873
logit 1.2249414224218522
output: tensor([-13.7452, -10.2398,  -8.3053,  -7.8855,  -7.4470,  -6.9832,  -6.4828,
         -5.9248,  -5.2628,  -4.3534,  -0.0252])
counts: tensor([  1,   1,   1,   1,   3,   3,   3,   4,  17,  54, 707])
(795,)
Optimization terminated successfully.
         Current function value: 0.097332
         Iterations: 0
         Function evaluations: 1
         Gradient evaluations: 1
mu 0.13836477987421394
theta 0.0029397849731835762
p 0.13795920946332138
logit 1.8323445324876688
output: tensor([-1.3528e+01, -1.0921e+01, -1.0326e+01, -9.2866e+00, -8.9789e+00,
        -8.5539e+00, -7.8426e+00, -1.5629e-03])
counts: tensor([  1,   1,   1

...

         Current function value: nan
         Iterations: 1
         Function evaluations: 112
         Gradient evaluations: 112
(795,)
Optimization terminated successfully.
         Current function value: 0.120161
         Iterations: 4
         Function evaluations: 13
         Gradient evaluations: 13
mu 0.03018867924516885
theta 0.03384985789612229
p 0.02920025477065172
logit 3.503942777160203
output: tensor([-1.3168e+01, -1.0246e+01, -7.6930e+00, -6.2361e-04])
counts: tensor([  1,   1,  15, 778])
(795,)
Optimization terminated successfully.
         Current function value: 0.009658
         Iterations: 0
         Function evaluations: 1
         Gradient evaluations: 1
mu 0.0012578616352201257
theta 20.0
p 5.989817310572027e-05
logit 9.722804652410682
output: tensor([-1.6403e+01, -7.5294e-08])
counts: tensor([  1, 794])
(795,)
 data for (contrl, control): None
(795,)
Optimization terminated successfully.
         Current function value: 0.083668
         Iterations: 0
         F

/Users/meiqiliu/miniforge3/envs/fa_base2/lib/python3.8/site-packages/statsmodels/base/model.py:592: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '
/Users/meiqiliu/miniforge3/envs/fa_base2/lib/python3.8/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/meiqiliu/miniforge3/envs/fa_base2/lib/python3.8/site-packages/statsmodels/discrete/discrete_model.py:3122: RuntimeWarning: invalid value encountered in log
  a1 * np.log(a1) + y * np.log(mu) -
/Users/meiqiliu/miniforge3/envs/fa_base2/lib/python3.8/site-packages/statsmodels/discrete/discrete_model.py:3123: RuntimeWarning: invalid value encountered in log
  (y + a1) * np.log(a2))
/Users/meiqiliu/miniforge3/envs/fa_base2/lib/python3.8/site-packages/statsmodels/discrete/discrete_model.py:3159:

         Current function value: 0.097456
         Iterations: 1
         Function evaluations: 124
         Gradient evaluations: 112
mu 0.045283018867924
theta 0.002937063290264211
p 0.04515040925835089
logit 3.0515544872340197
output: tensor([-1.3612e+01, -9.7522e+00, -8.9992e+00, -3.6964e-04])
counts: tensor([  1,   2,   8, 784])
(795,)
Optimization terminated successfully.
         Current function value: 0.369304
         Iterations: 4
         Function evaluations: 14
         Gradient evaluations: 14
mu 0.22641509433891216
theta 0.03258444326362047
p 0.2192702938883111
logit 1.2699238155500894
output: tensor([-13.6198,  -9.8255,  -8.3285,  -8.0803,  -7.8170,  -7.5338,  -7.2235,
         -6.8740,  -6.4627,  -5.9389,  -5.1434,  -0.0144])
counts: tensor([  1,   1,   1,   1,   1,   2,   3,   5,   4,  10,  23, 743])
(795,)
Optimization terminated successfully.
         Current function value: 0.017573
         Iterations: 0
         Function evaluations: 1
         Gradient evaluati

Optimization terminated successfully.
         Current function value: 0.486786
         Iterations: 2
         Function evaluations: 11
         Gradient evaluations: 11
mu 0.27295597484270834
theta 0.06175200539379608
p 0.2570807245534431
logit 1.0611972531448952
output: tensor([-15.6068,  -9.8168,  -8.4176,  -8.1151,  -7.8011,  -7.1245,  -6.7505,
         -6.3387,  -5.8675,  -5.2885,  -4.4513,  -0.0258])
counts: tensor([  1,   1,   1,   3,   1,   1,   3,   3,   4,  11,  51, 715])
(795,)
Optimization terminated successfully.
         Current function value: 0.009658
         Iterations: 0
         Function evaluations: 1
         Gradient evaluations: 1
mu 0.0012578616352201257
theta 20.0
p 5.989817310572027e-05
logit 9.722804652410682
output: tensor([-1.6403e+01, -7.5294e-08])
counts: tensor([  1, 794])
(795,)
Optimization terminated successfully.
         Current function value: 0.590289
         Iterations: 4
         Function evaluations: 9
         Gradient evaluations: 9
mu 0.3

/Users/meiqiliu/miniforge3/envs/fa_base2/lib/python3.8/site-packages/statsmodels/base/model.py:592: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '
/Users/meiqiliu/miniforge3/envs/fa_base2/lib/python3.8/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/var/folders/12/d54y_pk95zdbftjg_qkcj_c80000gq/T/ipykernel_42688/534514316.py:123: RuntimeWarning: invalid value encountered in log
  logit=np.log((1-p)/p)
/var/folders/12/d54y_pk95zdbftjg_qkcj_c80000gq/T/ipykernel_42688/534514316.py:123: RuntimeWarning: invalid value encountered in log
  logit=np.log((1-p)/p)
/Users/meiqiliu/miniforge3/envs/fa_base2/lib/python3.8/site-packages/statsmodels/discrete/discrete_model.py:3122: RuntimeWarning: invalid value encountered in log
  a1 * np.log(a1) + y * np

In [ ]:
def compute_log_zinb(x,const_x,alpha_x):
    mu=np.exp(const_x)
    print("mu",mu)
    theta=1/alpha_x
    p=1/(1+theta)*mu
    logit=np.log((1-p)/p)
    print("theta",theta)
    print("p",p)
    print("logit",logit)
    x_val=log_zinb_positive(torch.tensor(x[i]),torch.tensor(mu),torch.tensor(theta),torch.tensor(logit))
    most_frequent=torch.mode(x_val).values.item()
    return most_frequent 
    
    

def load_saved_data():
    list_stim=list(train.obs['perturbation'].unique())
    list_stim.remove('control')
    
    for stim in list_stim:
        x=np.loadtxt(str(stim)+'datax.csv', delimiter=',')
        y=np.loadtxt(str(stim)+'datay.csv', delimiter=',')
        df=pd.read_csv(str(stim)+'df.csv') 
        resx=[]
        resy=[]
        for i in range(x.shape[0]):
            if all(elem == 0 for elem in x[i]):
                continue 
            else:
                outputx=compute_log_zinb(x[i],df['const_x'],df['alpha_x'])
                outputy=compute_log_zinb(y[i],df['const_y'],df['alpha_y'])
                resx.append(outputx)
                resy.append(outputy)
        avgx=np.mean(resx)
        avgy=np.mean(resy)
        
        
            
            
        